This code analyses the performance of a trade system using the export of an account statement excel from ThinkorSwim.

In [13]:
import pandas as pd

# Excel dosyasını yükle
file_path = 'C:/Users/PC/Downloads/backtest_xcl.xlsx'
df = pd.read_excel(file_path, header=None)

# Sütun adlarını kontrol et ve doğru sütun adlarını kullan
#print(df.columns)

# Trade tipini (BOT veya SOLD) ve fiyatı çıkar
df['Type'] = df.iloc[:, 2].apply(lambda x: 'BOT' if 'BOT' in x else 'SOLD')
df['Price'] = df.iloc[:, 2].str.extract(r'@(\d+\.\d+)').astype(float)

# Gereksiz sütunları kaldır ve indexi sıfırla
df = df[['Type', 'Price']].reset_index(drop=True)

print(df)

# Hesaplamalar için değişkenleri başlat
total_profit_loss = 0
total_trades = 0
winning_trades = 0
losing_trades = 0
total_profit = 0
total_loss = 0

# Her BOT ve SOLD çifti için işlemleri yap
for i in range(0, len(df), 2):
    if i + 1 < len(df):
        bot_price = df.loc[i, 'Price']
        sold_price = df.loc[i+1, 'Price']

        profit_loss = sold_price - bot_price
        total_profit_loss += profit_loss

        if profit_loss > 0:
            winning_trades += 1
            total_profit += profit_loss
        else:
            losing_trades += 1
            total_loss += profit_loss

        total_trades += 1

# İstatistikleri hesapla
win_rate = winning_trades / total_trades if total_trades > 0 else 0
average_profit_per_trade = total_profit / winning_trades if winning_trades else 0
average_loss_per_trade = total_loss / losing_trades if losing_trades else 0
reward_to_risk = abs(average_profit_per_trade / average_loss_per_trade)
expected_profit_per_trade = win_rate * average_profit_per_trade - (1-win_rate) * average_loss_per_trade

# Sonuçları hazırla
results = {
    'Total Profit/Loss': total_profit_loss,
    'Total Trades': total_trades,
    'Winning Trades': winning_trades,
    'Losing Trades': losing_trades,
    'Win Rate': win_rate,
    'Average Profit per Trade': average_profit_per_trade,
    'Average Loss per Trade': average_loss_per_trade,
    'R:R Ratio': reward_to_risk,
    'Expected Profit per Trade': expected_profit_per_trade
}

# Sonuçları formatlı şekilde yazdır
for key, value in results.items():
    if isinstance(value, float):
        print(f'{key}: {value:.2f}')
    else:
        print(f'{key}: {value}')


     Type  Price
0     BOT  23.58
1    SOLD  19.24
2     BOT  17.86
3    SOLD  19.57
4     BOT  15.83
..    ...    ...
203  SOLD  27.10
204   BOT  28.30
205  SOLD  30.62
206   BOT  25.61
207  SOLD  28.10

[208 rows x 2 columns]
Total Profit/Loss: 49.26
Total Trades: 104
Winning Trades: 79
Losing Trades: 25
Win Rate: 0.76
Average Profit per Trade: 2.05
Average Loss per Trade: -4.50
R:R Ratio: 0.46
Expected Profit per Trade: 2.64
